## Step:1 Dataset Description and Objective:

**Dataset Description:** The dataset for this project, walmart.csv, represents weekly sales data from various Walmart outlets across the country. These retail stores, which have multiple outlets nationwide, are facing challenges in managing their inventory to effectively match supply with demand. The dataset contains 6435 rows and 8 columns. The columns represent the following features:

Store: The store number         
Date: The week of sales          
Weekly_Sales: The sales for the given store in that week          
Holiday_Flag: A flag indicating if it is a holiday week          
Temperature: The temperature on the day of the sale         
Fuel_Price: The cost of fuel in the region           
CPI: The Consumer Price Index          
Unemployment: The Unemployment Rate            

**Objective of the EDA:** The objective of the Exploratory Data Analysis (EDA) for this project is to gain insights into the factors affecting the weekly sales of the Walmart stores. The EDA will focus on understanding the impact of various factors such as unemployment rate, seasonal trends, temperature, and Consumer Price Index on the weekly sales. The EDA will also identify the top and worst performing stores based on the historical sales data. These insights will then be used to forecast the sales for each store for the next 12 weeks using predictive modeling techniques. The ultimate goal is to help Walmart manage its inventory more effectively by matching the demand with respect to supply.

## Step:2 Import Necessary libraries

In [ ]:
# Standard data manipulation libraries
import pandas as pd
import numpy as np
import math
from math import sqrt

# Data visualization libraries
!pip install plotly
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine learning libraries
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Ignore UserWarning
import warnings
warnings.filterwarnings("ignore")

# Evaluation Metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Library for Model Saving and Loading
import joblib

## Step:3 Data loading

In [ ]:
df = pd.read_csv(r"E:\csv\Intel csv\Walmart DataSet.csv")

## Step:4 Exploratory Data Analysis (EDA)

### 1) Data Overview
* Checking the dimensions of the dataset (number of rows and columns).
* Inspect first few rows to understand the structure of the data.

In [ ]:
# Display first 5 rows
df.head()

In [ ]:
# Display last 5 rows
df.tail()

In [ ]:
# checking shape of dataset
df.shape

* In this dataset we have 6435 rows and 8 columns

In [ ]:
# It helps to understand the data type and information about data
df.info()

From the info() method, we can deduce that our dataset encompasses three different data types: int64, float64, and object.

**Datatype of column:**
* The Store and Holiday_Flag columns are of type int64.          
* The Date column is categorized as object, typically indicative of string or categorical data. 
    * If the Date column represents dates, it might be advantageous to convert it to a datetime data type for facilitating date-related operations.        
* The remaining columns have a data type of float64.           

**About missing values:**
* By employing this method, we can effortlessly detect the presence of any missing values. 
* Here since each column contains 6435 observations, corresponding to the identical number of rows we previously observed using the shape attribute. So we can conclude that none of the column has missing values.

### 2) Checking for Duplicates:
* Identify and remove duplicate rows in the dataset to ensure data integrity.

In [ ]:
# Checking Duplicates Values
df.duplicated().sum()

* Sum is 0, meaning there are no duplicate rows in the DataFrame.

### 3) Summary Statisitc

In [ ]:
# The describe method Compute descriptive statistics for numerical variables like min,max etc
df.describe()

* Store Distribution: There are 45 unique stores represented in the dataset, ranging from Store 1 to Store 45. The distribution of data across stores is relatively balanced, with each store having a similar number of observations on average.

* Weekly Sales Distribution: The average weekly sales across all stores is approximately $1,046,965. The minimum and maximum weekly sales are approximately $209,986 and $3,818,686 respectively. This indicates a significant variation in weekly sales.

* Holiday Flag Distribution: Approximately 7% of the data points are from holiday weeks.

* Temperature Distribution: The average temperature is about 60.66°F, with a minimum of -2.06°F and a maximum of 100.14°F. The distribution of temperatures appears to be relatively normal.

* Fuel Price Distribution: The average fuel price is approximately $3.36 per gallon, with prices ranging from around $2.47 to $4.47 per gallon. The distribution of fuel prices appears to be relatively normal.

* Consumer Price Index (CPI) Distribution: The average CPI is approximately 171.58, with values ranging from approximately 126.06 to 227.23. The distribution of CPI values may vary across different regions and time periods.

* Unemployment Rate Distribution: The average unemployment rate is approximately 8%, with rates ranging from approximately 3.87% to 14.31%. The distribution of unemployment rates may reflect economic conditions and regional variations.

Overall, these insights provide a comprehensive overview of the distribution and variability of each feature in the dataset. Further analysis, such as correlation analysis and visualization, can help uncover additional patterns and relationships within the data.

In [ ]:
# Frequency counts for the 'Store' column
store_counts = df['Store'].value_counts()
store_counts

* So from above data we can tell that each store have 143 records, suggesting consistent data collection across all locations.

In [ ]:
# Frequency counts for the 'Holiday_Flag' column
holiday_counts = df['Holiday_Flag'].value_counts()
holiday_counts

* There are 5985 instances (or weeks) where Holiday_Flag is 0, indicating regular weeks without any holidays. 
* Additionally, there are 450 instances where Holiday_Flag is 1, signifying these weeks include at least one holiday

### 4) Handling missing values

In [ ]:
# Checking if any column having null values
df.isnull().sum()

* Previously, we determined that our dataset has no missing values by using the info() method. We reverified this by using the isna() function, confirming that our dataset does not contain any null values.
* We also checked for any irregularities in our data and found none.

### 5) Outlier Analysis

In [ ]:
import plotly.express as px

col = df.columns.values

for col_name in col:
    if df[col_name].dtype != 'object':
        fig = px.box(df, y=col_name)
        fig.update_layout(title=f'Box plot of {col_name}', xaxis_title=col_name, yaxis_title='Count')
        fig.show()

 **Note:1**

* For categorical variables with only two unique values (often represented as 1 and 0) like here in 'Holiday_Flag', the concept of outliers doesn't apply in the traditional sense. Outliers are typically defined as data points that significantly deviate from the rest of the observations in a numerical distribution.
* so we have outliers in 'Weekly_sales', 'Temperature' and 'Unemployment' columns

 **Note:2**
* In many cases, 'Weekly_Sales' is considered as the target or dependent variable in predictive modeling tasks, such as sales forecasting.
* Removing outliers from the target variable may lead to biased model predictions. 
* If outliers represent genuine data points (e.g., exceptionally high sales during holiday seasons), removing them can distort the model's ability to capture such patterns.
* However, it's essential to preprocess other independent variables (such as 'Temperature' and 'Unemployment') to mitigate the impact of outliers on model training and performance.

 **Note:3**
* Here ‘Temperature’ and ‘Unemployment’ columns have outliers, it could be due to natural fluctuations in weather and economic conditions. 
* In such cases, these outliers are actual representations of the variability in the data, and removing them might lead to loss of information.
* I have checked before if we remove these outliers 
    * So before removing outlier, we have 6435 rows
    * but after removing outlier we got only 5951 rows, 
    * means we have removed 484 rows and i.e. nothing but loss of 7.5% of data and i.e not a small amount. which could potentially include important, meaningful data. 
* Therefore i am thinking filling data with median is good way to resolve this issue.

Imputation: Replace the outlier values with statistical measures such as mean, median or mode.

In [ ]:
import plotly.express as px

# Define column names
col_names = ['Temperature', 'Unemployment']

# Iterate over each column
for col_name in col_names:
    # Calculate quartiles and IQR
    Q1 = df[col_name].quantile(0.25)
    Q3 = df[col_name].quantile(0.75)
    IQR = Q3 - Q1
    
    # Define lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Replace outliers with the median of the column
    df[col_name] = df[col_name].apply(lambda x: df[col_name].median() if (x < lower_bound or x > upper_bound) else x)

# Create box plots after replacing outliers
for col_name in col_names:
    fig = px.box(df, y=col_name)
    fig.update_layout(title=f'Box plot of {col_name} (Outliers Replaced)', xaxis_title=col_name, yaxis_title='Value')
    fig.show()

### 6) Univariate Analysis

#### 6.1) Examine the distribution of Variables using histograms

* These columns are particularly relevant for understanding the factors that influence weekly sales and overall store performance. By visualizing their distributions through histograms, we can gain insights into the variability, central tendency, and potential relationships within the data, aiding in the analysis and decision-making process for inventory management strategies.

* Weekly_Sales: This is the target variable, and understanding its distribution is crucial for analyzing sales patterns across different stores and time periods. A histogram of weekly sales can provide insights into the range of sales values, the presence of outliers, and the overall distribution shape.

* Temperature: Analyzing the distribution of temperature values can help identify seasonal patterns and understand how temperature variations might impact sales of certain products, such as seasonal items or weather-dependent goods.

* Fuel_Price: Examining the distribution of fuel prices can reveal trends in fuel costs over time and their potential impact on transportation expenses, which may influence sales patterns and store profitability.

* CPI (Consumer Price Index): Understanding the distribution of CPI values can provide insights into regional economic conditions and consumer spending power, which may affect purchasing behavior and sales trends.

* Unemployment: Analyzing the distribution of unemployment rates can help assess the economic environment in different regions and understand how employment trends might influence consumer confidence and spending patterns.

In [ ]:
# Distribution of Weekly_Sales
fig = px.histogram(df, x = df['Weekly_Sales'], marginal='box', title = 'Distribution of Weekly_Sales', nbins = 50)
fig.update_layout(xaxis_title = 'Weekly_Sales', yaxis_title = 'Frequency')
fig.show()

* The distribution of weekly sales is right-skewed, meaning there are a few weeks with exceptionally high sales.
* The majority of the weekly sales values fall within the box in the box plot, while there are some outliers represented as individual points above the box.
* The most common range of weekly sales is around 0.5M, as indicated by the highest bar in the histogram.

In [ ]:
# Distribution of Temperature
fig = px.histogram(df, x = df['Temperature'],marginal='box', title = 'Distribution of Temperature', nbins = 50)
fig.update_layout(xaxis_title = 'Temperature', yaxis_title = 'Frequency')
fig.show();

* The shape of the distribution appears to be somewhat bell-shaped, peaking around a temperature of 60. This suggests that most of the temperature values in your dataset are around this value.
* There are fewer occurrences of very low and very high temperatures, as indicated by the lower bars at the ends of the histogram.

In [ ]:
# Distribution of Fuel_Price
fig = px.histogram(df, x = df['Fuel_Price'],marginal='box', title = 'Distribution of Fuel_Price', nbins = 50)
fig.update_layout(xaxis_title = 'Fuel_Price', yaxis_title = 'Frequency')
fig.show()

* The shape of the distribution appears to be somewhat bell-shaped, peaking around a fuel price of 3.75 to 3.799. This suggests that most of the fuel prices in your dataset are around this value.
* The histogram shows that the frequency of fuel prices between 2.5 and 3.5 is relatively lower compared to the peak around 3.75 to 3.799.
* There are fewer occurrences of very low and very high fuel prices, as indicated by the lower bars at the ends of the histogram.

In [ ]:
# Distribution of CPI
fig = px.histogram(df, x = df['CPI'], marginal='box', title = 'Distribution of CPI', nbins = 50)
fig.update_layout(xaxis_title = 'CPI', yaxis_title = 'Frequency')
fig.show()

* The shape of the distribution appears to be bimodal, with two peaks: one around a CPI of 140 and another just above 200. 
* This suggests that most of the CPI values in your dataset are around these two values.
* There are fewer occurrences of very low and very high CPI values, as indicated by the lower bars at the ends of the histogram.

In [ ]:
# Distribution of Unemployment
fig = px.histogram(df, x = df['Unemployment'], marginal='box', title = 'Distribution of Unemployment', nbins = 50)
fig.update_layout(xaxis_title = 'Unemployment', yaxis_title = 'Frequency')
fig.show()

* The shape of the distribution appears to be somewhat bell-shaped, peaking around an unemployment rate of around 8. This suggests that most of the unemployment rates in your dataset are around this value.
* There are fewer occurrences of very low and very high unemployment rates, as indicated by the lower bars at the ends of the histogram.

    In summary:
* Weekly_Sales is right-skewed
* Temperature, Fuel Price, and Unemployment Rate appear to be somewhat normally distributed
* CPI appears to be bimodal, indicating that there are two distinct peaks or modes in the distribution

#### 6.2) Feature Engineering

In [ ]:
df.info()

In [ ]:
df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y")

* During info() method we have seen 'Date' column is of object type, !!!Right
* but as we are planning to perform forecasting, it is important to have our ‘Date’ column in a standard date format.

In [ ]:
df.info()

So now, we have `one column of 'datetime' data type,` and `the remaining columns are either of 'int64' or 'float64' data types`, which represent numerical values. Therefore, there is no need for label or one-hot encoding to convert categorical variables into numerical format

In [ ]:
df.head(2)

In [ ]:
import calendar

def get_season(month):
    # Define North American seasons based on month ranges
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

# Use the 'assign' method to add multiple columns in a single line
df = df.assign(
    Day=df['Date'].dt.day,  # Add a day column
    Day_Of_Week=df['Date'].dt.day_name(),  # Add a day_of_week column
    Week=df['Date'].dt.isocalendar().week,  # Add a week column
    Month=df['Date'].dt.month,  # Add a month column
    Month_Name=df['Date'].dt.month_name(),  # Add a month_name column
    Quarter=df['Date'].dt.quarter,  # Add a quarter column (q1, q2, q3 and q3)
    Season=df['Date'].dt.month.apply(get_season),  # Assign seasons based on month
    Year=df['Date'].dt.year  # Add a year column
)

In [ ]:
df.head(2)

In [ ]:
df.info()

#### 6.3) Analyzing categorical variable
* Categorical Features such as Holiday_Flag, Day_Of_Week, Month_Name, Season

In [ ]:
# Compare weekly sales between holidays and non-holidays
fig = px.box(df, x='Holiday_Flag', y='Weekly_Sales', title='Weekly Sales by Holiday', width = 800, height = 500)
fig.update_layout(xaxis=dict(tickmode='linear', tickvals=[0, 1], ticktext=['Normal Day', 'Holiday']))
fig.show()

* Avergae sale are higher during holidays as compared to non-holiday days

In [ ]:
# Create a pie chart
labels = ['Not Holiday', 'Holiday']
values = df['Holiday_Flag'].value_counts()
colors = ['#FF6692', '#FFDBC0']  # Using predefined color names

fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='percent', marker=dict(colors=colors))])
fig.update_layout(title='Distribution of Holiday Flag')
fig.show()

* By this pie chart we can cleary see 'Dominance of Non-Holiday Sales' means bulk of sales occurs during non-holiday periods
* But 6.99% sales done during holiday smaller in percentage, yet potentially significant, increase in sales during holidays. This could be due to seasonal promotions or holiday shopping trends. 
* Strategic Planning: The distribution suggests that while holiday periods are important, the store should focus on optimizing sales throughout the year, as non-holiday periods contribute more to the overall sales volume.

In [ ]:
# Distribution of Observations by Year:
year_counts = df['Year'].value_counts()

# Create a bar plot
fig = go.Figure(data=[go.Bar(
    x=year_counts.index.astype(str),  # Years
    y=year_counts.values, # Counts
    text=year_counts.values,  # Text displayed on each bar
    textposition='auto',  # Automatically position text
    marker=dict(color=['#F08080', '#ADD8E6', '#90EE90'])  # Color palette
)])

# Customize layout
fig.update_layout(
    title='Count of Observations by Year',
    xaxis=dict(title='Year'),
    yaxis=dict(title='Count'),
)

# Show the plot
fig.show()

In [ ]:
# Distribution of Observations by Year:
year_counts = df['Year'].value_counts()

# Create a pie chart
fig = go.Figure(data=[go.Pie(labels=year_counts.index, values=year_counts, textinfo='percent', marker=dict(colors=['#F08080', '#ADD8E6', '#90EE90']))])

# Customize layout
fig.update_layout(title='Distribution of Observations by Year')

# Show the plot
fig.show()

* The bar plot and pie chart illustrates an even distribution of observations across 2010, 2011, and 2012. 
* Though 2011 shows a slight increase, overall consistency suggests stable trends over the three years.

In [ ]:
# Distribution of Observations by Season:
season_counts = df['Season'].value_counts()

# Create a bar plot
fig = go.Figure(data=[go.Bar(
    x=season_counts.index,  # Seasons
    y=season_counts.values,  # Counts
    text=season_counts.values,  # Text displayed on each bar
    textposition='auto',  # Automatically position text
    marker=dict(color=['#FA6E1B', '#008000','#FFA500', '#A6D1F3'])  # Color palette
)])

# Customize layout
fig.update_layout(
    title='Count of Observations by Season',
    xaxis=dict(title='Season'),
    yaxis=dict(title='Count'),
)

# Show the plot
fig.show()


* Summer has the highest count of observations, suggesting that it might be the busiest season.
* Spring follows closely behind Summer in terms of the count of observations.
* Fall and Winter have fewer observations, indicating that these seasons might be less busy.

In [ ]:
# Distribution of Observations by Season:
season_counts = df['Season'].value_counts()

# Create a pie chart
fig = go.Figure(data=[go.Pie(labels=season_counts.index, values=season_counts, textinfo='percent', marker=dict(colors=['#FA6E1B', '#008000','#FFA500', '#A6D1F3']))])

# Customize layout
fig.update_layout(title='Distribution of Observations by Season')

# Show the plot
fig.show()

This pie chart provides a visual representation of the distribution of observations across different seasons. Here are some insights we can get from this chart

* Summer Dominance: With the largest segment at 28%, summer appears to be the season with the highest number of observations, indicating it might be the busiest or most active time of the year for sales.
* Spring Significance: Spring is close behind summer, accounting for 27.3% of the observations, suggesting that it is also a significant time for sales.
* Fall and Winter: Fall, represented by 23.8%, and winter, at 21%, have fewer observations, which could imply a decrease in  sales during these seasons.

In [ ]:
# Distribution of Observations by Quarter:
year_counts = df['Quarter'].value_counts()

# Create a bar plot
fig = go.Figure(data=[go.Bar(
    x=year_counts.index,  # Quarter
    y=year_counts.values, # Counts
    text=year_counts.values,  # Text displayed on each bar
    textposition='auto',  # Automatically position text
    marker=dict(color=['#F08080', '#ADD8E6', '#90EE90', '#BC7FCD'])  # Color palette
)])

# Customize layout
fig.update_layout(
    title='Count of Observations by Quarter',
    xaxis=dict(title='Quarter'),
    yaxis=dict(title='Count'),
)

# Show the plot
fig.show()

The barplot graphically represents the counts of observations by quarter. Quarter 3 exhibits the highest count of observations, closely followed by Quarter 2 and Quarter 1, while Quarter 4 demonstrates the lowest count.

In [ ]:
# Get the counts of observations by Quarter
year_counts = df['Quarter'].value_counts()

# Create a pie chart
fig = go.Figure(data=[go.Pie(labels=year_counts.index, values=year_counts, textinfo='percent', marker=dict(colors=['#F08080', '#ADD8E6', '#90EE90', '#BC7FCD']))])

# Customize layout
fig.update_layout(title='Distribution of Observations by Quarter')

# Show the plot
fig.show()

The piechart illustrates the percentage distribution of observations across quarters. Notably, Quarter 3 has the highest percentage of observations at 28%, suggesting a potential increase in activity or sales during this period. Quarter 1 closely follows with 23.1%, while Quarters 2 and 4 exhibit lower percentages of observations at 27.3% and 21.7%, respectively.which helps to do strategic planning for resource allocation throughout the year.

In [ ]:
# Distribution of Observations by Month:
month_counts = df['Month_Name'].value_counts()

# Create a bar plot
fig = go.Figure(data=[go.Bar(
    x=month_counts.index,  # Months
    y=month_counts.values,  # Counts
    text=month_counts.values,  # Text displayed on each bar
    textposition='auto',  # Automatically position text
)])

# Customize layout
fig.update_layout(
    title='Count of Observations by Month',
    xaxis=dict(title='Month'),
    yaxis=dict(title='Count'),
)

# Rotate x-axis labels for better readability
fig.update_layout(xaxis_tickangle=-45)

# Show the plot
fig.show()

April and July emerge as peak sales months with 630 observations each, while March to October maintain steady activity around 585. However, November and January show notable declines to 360, possibly due to seasonal or post-holiday effects.

### 7) Bivariate Analysis:

In [ ]:
# Mean of Weekly Sales for Each Holiday Flag:
df.groupby('Holiday_Flag')['Weekly_Sales'].mean().reset_index()

**7.1 Mean of Weekly Sales for Each Holiday Flag:**

In [ ]:
# Calculate the mean weekly sales for each holiday flag
mean_sales = df.groupby('Holiday_Flag')['Weekly_Sales'].mean().reset_index()

# Create the bar plot
fig = px.bar(mean_sales, x='Holiday_Flag', y='Weekly_Sales', 
             title='Average Weekly Sales by Holiday Flag',
             labels={'Holiday_Flag': 'Is Holiday', 'Weekly_Sales': 'Average Weekly Sales'})

# Show the plot
fig.show()

* So from above plot we can easily deduce that higher average sales during holiday periods (1 on x-axis) compared to non-holidays (0), suggesting a positive impact of holidays on sales due to increased consumer spending. This insight enables businesses to strategize inventory and marketing efforts to capitalize on holiday shopping seasons, reflecting a common trend of sales spikes during holidays driven by promotions and festive shopping behaviors.

In [ ]:
df.groupby('Holiday_Flag')['Weekly_Sales'].sum().reset_index()

**7.2 Total Weekly Sales for Each Holiday Flag:**

In [ ]:
# Calculate the total weekly sales for each holiday flag
total_sales = df.groupby('Holiday_Flag')['Weekly_Sales'].sum().reset_index()
# Create the bar plot
fig = px.bar(total_sales, x='Holiday_Flag', y='Weekly_Sales', 
             title='Total Sales by Holidays',
             labels={'holiday_flag': 'Is Holiday', 'weekly_sales': 'Total Sales'})

# Show the plot
fig.show()

Total sales are higher during non-holiday periods, although holidays also see a sales spike. Businesses can use this data to strategize inventory and marketing, focusing on both holiday and non-holiday periods. 

In [ ]:
df.groupby('Store')['Weekly_Sales'].sum().sort_values(ascending=False).reset_index()

**7.3 Sum of Weekly Sales for Each Store:**

In [ ]:
# Calculate the sum of weekly sales for each store.
total_sales_per_store = df.groupby('Store')['Weekly_Sales'].sum().reset_index()

# Create the bar plot
fig = px.bar(total_sales_per_store, x='Store', y='Weekly_Sales', 
             title='Total Sales in each Store',
             labels={'Store': 'Store', 'Weekly_Sales': 'Total Sales'})

# Set the figure size
fig.update_layout(
    width=900,
    height=500
)

# Show the plot
fig.show()

* Top Performing Stores: Stores 20, 4, 14, and 13 are identified as the top performing stores based on total sales. These stores have demonstrated consistently high sales figures.

* Worst Performing Stores: Stores 36, 5, 44, and 33 are identified as the worst performing stores based on total weekly sales. These stores have notably lower sales figures compared to others.

So this bar graph shows a significant variation in weekly sales across different stores. Store No. 20 leads with the highest sales of 301,397,792, closely followed by Store No. 4 with sales of 299,543,953. In contrast, Store No. 33 has the lowest sales, amounting to 37,160,222. This suggests differing performance levels among the stores and potential areas for improvement, especially for those with lower sales

**7.4) Scatter Plot Analysis**

Because Weekly_Sales is influenced by Temperature, Fuel_Price, CPI, Unemployment, and Holiday_Flag, it's essential to draw scatter plots to understand the relationship between these columns and Weekly_Sales:

* Temperature vs Weekly_Sales
* Fuel_Price vs Weekly_Sales
* CPI vs Weekly_Sales
* Unemployment vs Weekly_Sales

**7.4.1 Temperature vs Weekly Sales:**

In [ ]:
fig = px.scatter(df, x='Temperature', y='Weekly_Sales', title='Temperature vs Weekly Sales', height = 400, width = 800)
fig.show()

* There appears to be a moderate positive correlation between temperature and weekly sales, indicating that higher temperatures may lead to increased sales.

**7.4.2 Scatter plot for Fuel_Price vs Weekly_Sales:**

In [ ]:
fig = px.scatter(df, x='Fuel_Price', y='Weekly_Sales', title='Fuel Price vs Weekly Sales')
fig.show()

*  There is no clear trend between fuel price and weekly sales, suggesting that fuel price may not have a significant impact on sales.

**7.4.3 Scatter plot for CPI vs Weekly_Sales:**

In [ ]:
fig = px.scatter(df, x='CPI', y='Weekly_Sales', title='CPI vs Weekly Sales')
fig.show()

* Consumer Price Index (CPI) has little impact on sales. Based on the distribution of typical consumer prices in the figure above, clients can be categorized into two groups: clients that pay from 120 and 150 are considered middle-class clients. consumers who pay between 180 and 230 are considered high-class consumers.

**7.4.4 Scatter plot for Unemployment vs Weekly_Sales:**

In [ ]:
fig = px.scatter(df, x='Unemployment', y='Weekly_Sales', title='Unemployment vs Weekly Sales')
fig.show()

* There appears to be a slight negative correlation between unemployment rate and weekly sales, indicating that lower unemployment rates may be associated with higher sales.

**7.5 Analysis of Seasonal Trends in Weekly Sales**

Identifying seasonal trends in weekly sales involves analyzing the data to observe patterns that repeat over specific periods, such as weeks, months, or quarters.

**7.5.1 Total sales in each year:**

In this analysis, we examine the total sales in each year to identify any significant trends or fluctuations.

In [ ]:
# Group data by year and calculate total sales
sales_by_year = df.groupby('Year')['Weekly_Sales'].sum().reset_index()

# Create bar plot
fig = go.Figure(data=go.Bar(
    x=sales_by_year['Year'],
    y=sales_by_year['Weekly_Sales'],
))

# Update layout
fig.update_layout(
    title='Total Sales in each Year',
    xaxis=dict(title='Year'),
    yaxis=dict(title='Total Sales'),
    width=750,
    height=500,
)

# Show plot
fig.show()

In [ ]:
# Total sales in each year

# Group data by year and calculate total sales
sales_by_year = df.groupby('Year')['Weekly_Sales'].sum().reset_index()

# Create line plot
fig = go.Figure(data=go.Scatter(
    x=sales_by_year['Year'],
    y=sales_by_year['Weekly_Sales'],
    mode='lines+markers',
    marker=dict(color='blue', size=8),
    line=dict(color='blue', width=2),
))

# Update layout
fig.update_layout(
    title='Total Sales in each Year',
    xaxis=dict(title='Year'),
    yaxis=dict(title='Total Sales'),
    width=750,
    height=500,
)

# Show plot
fig.show()

* In 2011, total sales peaked at $2.4482 billion, but there was a significant drop in 2012. This suggests that certain factors, potentially specific events, marketing campaigns, or economic conditions, might have influenced the sales performance. 
* It’s crucial to investigate these variables to understand the decline in 2012 and strategize effectively for future sales growth.

**7.5.2 Total Sales in each Season:**

This analysis examines the total sales in each season to understand the seasonal trends in sales performance.

In [ ]:
# Total Sales in each Season
# Group data by season and calculate total sales
sales_by_season = df.groupby('Season')['Weekly_Sales'].sum().reset_index()

# Define color palette for each season
colors = ['#FA6E1B', '#008000', '#FFA500', '#A6D1F3']

# Create bar plot
fig = go.Figure(go.Bar(
    x=sales_by_season['Season'],
    y=sales_by_season['Weekly_Sales'],
    marker_color=colors
))

# Update layout
fig.update_layout(
    title='Total Sales in each Season',
    xaxis=dict(title='Season'),
    yaxis=dict(title='Total Sales'),
)

# Show plot
fig.show()

In [ ]:
# Group data by season and calculate total sales
sales_by_season = df.groupby('Season')['Weekly_Sales'].sum().reset_index()

# Create line plot
fig = go.Figure(go.Scatter(
    x=sales_by_season['Season'],
    y=sales_by_season['Weekly_Sales'],
    mode='lines+markers',
    marker=dict(color='blue', size=8),
    line=dict(color='blue', width=2),
))

# Update layout
fig.update_layout(
    title='Total Sales in each Season',
    xaxis=dict(title='Season'),
    yaxis=dict(title='Total Sales'),
)

# Show plot
fig.show()

* So this graph indicates that Spring and Summer seasons experience the highest sales, which could be attributed to seasonal shopping habits, such as purchasing for holidays or outdoor activities. 
* Despite slight variations, total sales appear relatively consistent across all seasons, suggesting a stable customer base and steady demand throughout the year.
* The lower sales in Fall and Winter could prompt Walmart to consider marketing strategies or promotions to boost sales during these seasons.

**7.5.3 Total Sales in each Quarter:**

In [ ]:
# Group data by Quarter and calculate total sales
sales_by_month = df.groupby('Quarter')['Weekly_Sales'].sum().reset_index()

# Create bar plot
fig = go.Figure(go.Bar(
    x=sales_by_month['Quarter'],
    y=sales_by_month['Weekly_Sales']
))

# Update layout
fig.update_layout(
    title='Total Sales in each Quarter',
    xaxis=dict(title='Quarter'),
    yaxis=dict(title='Total Sales'),
    showlegend=False
)

# Show plot
fig.show()

In [ ]:
import plotly.graph_objects as go

# Group data by quarter and calculate total sales
sales_by_quarter = df.groupby('Quarter')['Weekly_Sales'].sum().reset_index()

# Create line plot
fig = go.Figure(go.Scatter(
    x=sales_by_quarter['Quarter'],
    y=sales_by_quarter['Weekly_Sales'],
    mode='lines+markers',
    marker=dict(color='blue', size=8),
    line=dict(color='blue', width=2),
))

# Update layout
fig.update_layout(
    title='Total Sales in each Quarter',
    xaxis=dict(title='Quarter'),
    yaxis=dict(title='Total Sales'),
    showlegend=False
)

# Show plot
fig.show()

* This bar graph shows that sales in Quarters 2 and 3 are the highest, suggesting a mid-year peak in shopping activity, possibly due to summer holidays and back-to-school shopping. 
* Quarters 1 and 4 have lower sales, with Quarter 1 being the lowest, which may reflect post-holiday spending reductions and end-of-year budget constraints. 
* These insights are helpful in optimizing inventory and marketing strategies for different times of the year

**7.5.4 Total Sales in each Week:**

In [ ]:
grp_week = df.groupby('Week')['Weekly_Sales'].sum().sort_values(ascending=False)
grp_week


# Create a bar plot
fig = go.Figure(go.Bar(
    x=grp_week.index,
    y=grp_week.values,
))

# Update layout
fig.update_layout(
    title='Total Sales in each Week',
    xaxis=dict(title='Week'),
    yaxis=dict(title='Total Sales'),
    bargap=0.2,  # Set the gap between bars
)

# Show plot
fig.show()


In [ ]:
import plotly.graph_objects as go

# Group data by week and calculate total sales
grp_week = df.groupby('Week')['Weekly_Sales'].sum().sort_index()

# Create a line plot
fig = go.Figure(go.Scatter(
    x=grp_week.index,
    y=grp_week.values,
    mode='lines+markers',
    line=dict(color='blue', width=2),
    marker=dict(color='blue', size=8),
))

# Update layout
fig.update_layout(
    title='Total Sales in each Week',
    xaxis=dict(title='Week'),
    yaxis=dict(title='Total Sales'),
)

# Show plot
fig.show()


The graph shows that Walmart’s weekly sales start strong, dip sharply around week 10, stabilize with minor fluctuations until week 40, and then exhibit significant volatility with sharp declines and spikes towards week 50. This pattern could reflect various external factors such as holidays, promotions, or seasonal changes affecting consumer behavior and sales trends.

**7.5.5 Visualize Weekly Sales Over Time:**

In [ ]:
fig = go.Figure(go.Bar(
    x=df['Date'],
    y=df['Weekly_Sales'],
    marker_color='skyblue'
))

# Update layout
fig.update_layout(
    title='Weekly Sales Over Time',
    xaxis=dict(title='Date', tickangle=45),
    yaxis=dict(title='Weekly Sales'),
    plot_bgcolor='rgba(0,0,0,0)',
    showlegend=False
)

# Show plot
fig.show()

In [ ]:
# visualize weekly sales over time
# Create a line plot

fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Date'], y=df['Weekly_Sales'],
                    mode='lines',
                    name='Weekly Sales'))

# Update layout
fig.update_layout(title='Weekly Sales Over Time',
                   xaxis_title='Date',
                   yaxis_title='Weekly Sales',
                   xaxis=dict(tickangle=45),
                   yaxis=dict(gridcolor='lightgray'),
                   showlegend=True)

# Show plot
fig.show()

According to line plit we can say that from February 2010 to October 2012 shows that weekly sales have been relatively stable, with two significant spikes in sales occurring around January 2011 and January 2012. These spikes likely correspond to seasonal events or promotions that drove higher sales volumes. The consistent fluctuation between approximately 30M and 50M for most of the time suggests a steady demand, with occasional increases potentially tied to specific high-traffic events or holidays. This data can be crucial for planning inventory and sales strategies to capitalize on peak times and maintain a steady supply during regular demand periods.

**7.5.6 Weekly_sales over the years:**

In [ ]:
# Create a pivot table
weekly_sales = df.pivot_table(values='Weekly_Sales', columns='Year', index='Week')

# Create a line plot for each year
fig = go.Figure()
for year in weekly_sales.columns:
    fig.add_trace(go.Scatter(x=weekly_sales.index, y=weekly_sales[year], mode='lines', name=str(year)))

# Update layout
fig.update_layout(
    title='Weekly Sales Over the Years',
    xaxis=dict(title='Week'),
    yaxis=dict(title='Weekly Sales'),
    legend=dict(title='Year'),
)

# Show plot
fig.show()


Conclusion:Total sales for all years in week 51 are the highest from any week, with 157,929,657$

**7.5.7 Total Sales for each Season in each Year:**

In [ ]:
# Group data by year and season and calculate total sales
sales_by_year_season = df.groupby(['Year', 'Season'])['Weekly_Sales'].sum().reset_index()

# Create a bar plot
fig = px.bar(sales_by_year_season, x='Year', y='Weekly_Sales', color='Season', barmode='group',
             title='Total Sales for each Season in each Year',
             labels={'Year': 'Year', 'Weekly_Sales': 'Total Sales', 'Season': 'Season'})

# Show plot
fig.show()


**Seasonal Sales Trends:** The graph shows that sales are highest in the Fall season each year, which could be due to back-to-school shopping or holiday sales events.          
**Spring Sales Dip:** There was a noticeable drop in sales during the Spring of 2011, which may require further investigation to understand the cause.        
**Consistent Winter Sales:** Sales during Winter have been consistent over the three years, indicating stable consumer behavior during this season.            
**Growing Summer Sales:** There is an upward trend in Summer sales from 2010 to 2012, suggesting increasing consumer activity or successful marketing strategies in that period.           

These insights can help in understanding consumer behavior and planning inventory and marketing strategies accordingly.

**7.5.8 Total Sales for each Month in each year**

In [ ]:
# Group data by year and month and calculate total sales
sales_by_year_month = df.groupby(['Year', 'Month_Name'])['Weekly_Sales'].sum().reset_index()

# Create a bar plot
fig = px.bar(sales_by_year_month, x='Year', y='Weekly_Sales', color='Month_Name', barmode='group',
             title='Total Sales for each Month in each Year',
             labels={'Year': 'Year', 'Weekly_Sales': 'Total Sales', 'Month_Name': 'Month'})

# Show plot
fig.show()

Conclusion:
* Monthly Sales Trends: The graph shows trends in total sales for each month over the years 2010, 2011, and 2012, highlighting which months had higher sales and allowing comparison of annual growth or decline.
* Consistent High-Performing Months: April and December consistently show higher total sales compared to other months across all three years, suggesting these months may have key events or holidays that drive sales.
* Sales Variability: The fluctuating monthly sales indicate that certain factors, possibly including seasonal changes, holidays, or promotions, significantly impact sales.
* No Clear Growth Pattern: The absence of a consistent pattern of growth or decline suggests that sales are influenced by a variety of factors, rather than a steady market change.
These insights can inform strategic decisions for inventory management, marketing campaigns, and sales forecasting.

### 8) Multivariate Analysis:
Now we are going to understand the relationship between all the different columns numerically to check how they correlate with the weekly sales in order to confirm the inferences we have gathered from the above EDA study.

In [ ]:
import plotly.graph_objects as go

# Calculate correlation matrix for numeric columns only
correlation_matrix = df.corr(numeric_only=True)

# Create heatmap for all columns
fig = go.Figure(data=go.Heatmap(
                    z=correlation_matrix, 
                    x=correlation_matrix.columns,
                    y=correlation_matrix.columns,
                    colorscale='RdYlBu',  
                    ))

fig.update_layout(title='Correlation Heatmap')
fig.show()

The correlation matrix offers valuable insights into how different variables may influence weekly sales at Walmart. Here’s a summary of the insights:

* Date and Fuel Price: There’s a strong positive correlation (0.77), suggesting that as time progresses, fuel prices also tend to increase.
* Weekly Sales and Store: A negative correlation (-0.34) indicates that higher store numbers might be associated with lower weekly sales, which could suggest a pattern in store performance based on their numbering.
* Weekly Sales and Unemployment: The negative correlation (-0.074) suggests that higher unemployment rates might be associated with slightly lower weekly sales.
* CPI: The Consumer Price Index doesn’t show significant correlations with Holiday_Flag or Temperature, indicating that CPI’s impact on weekly sales might be independent of these factors.

These insights can help Walmart understand the dynamics affecting sales and make informed decisions on inventory management, pricing, and promotions. However, it’s important to remember that correlation does not imply causation, and further analysis would be needed to establish direct relationships between these variables.

### 9)Data Correlation

Correlation: A measure of the extent of interdependence between variables.

Pearson Correlation: The Pearson Correlation measures the linear dependence between two variables X and Y.

* The resulting coefficient is a value between -1 and 1 inclusive, where:

* 1: Perfect positive linear correlation. 
* 0: No linear correlation, the two variables most likely do not affect each other. 
* -1: Perfect negative linear correlation.
<br>

* P-value: What is this P-value? The P-value is the probability value that the correlation between these two variables is statistically significant. Normally, we choose a significance level of 0.05, which means that we are 95% confident that the correlation between the variables is significant.

    * By convention, when the
    * p-value is < 0.001: we say there is strong evidence that the correlation is significant.
    * p-value is < 0.05: there is moderate evidence that the correlation is significant.
    * p-value is < 0.1: there is weak evidence that the correlation is significant.
    * p-value is > 0.1: there is no evidence that the correlation is significant.

**9.1 Pearson Correlation Coefficient and P-value of 'Fuel_Price' and 'Weekly_Sales':**

In [ ]:
 #Let's calculate the Pearson Correlation Coefficient and P-value of 'fuel_price' and 'weekly_sales':
from scipy.stats import pearsonr

# Assuming you have a DataFrame 'data' with columns 'fuel_price' and 'weekly_sales'
pearson_coef, p_value = pearsonr(df['Fuel_Price'], df['Weekly_Sales'])
print("The Pearson Correlation Coefficient is", pearson_coef, "with a P-value of P =", p_value)

In [ ]:
# Create a scatter plot with regression line
fig = px.scatter(df, x='Fuel_Price', y='Weekly_Sales', trendline='ols', 
                 title='Weekly Sales vs. Fuel Price', 
                 labels={'fuel_price': 'Fuel Price', 'weekly_sales': 'Weekly Sales'})

# Update trendline color
fig.data[1].line.color = 'red'

# Show plot
fig.show()

Conclusion:

Since the p-value is > 0.1, the correlation between fuel price and weekly sales is not statistically significant.
Fuel price does not seem like a good predictor of the weekly sales at all since the regression line is close to horizontal. Therefore, it's not a reliable variable

**9.2 Unemployment vs. Weekly Sales:**

In [ ]:
pearson_coef, p_value = pearsonr(df['Unemployment'], df['Weekly_Sales'])
print("The Pearson Correlation Coefficient is", pearson_coef, "with a P-value of P =", p_value)

In [ ]:
# Create a scatter plot with regression line
fig = px.scatter(df, x='Unemployment', y='Weekly_Sales', trendline='ols', 
                 title='Weekly Sales vs. Unemployment Rate', 
                 labels={'unemployment': 'Unemployment Rate', 'weekly_sales': 'Weekly Sales'})

# Update trendline color
fig.data[1].line.color = 'red'

# Show plot
fig.show()

Conclusion:-

Since the p-value is < 0.001, the correlation between unemployment and weekly sales is strong evidence that the correlation is significant.
Unemployment seems like a good predictor of the weekly sales, The higher the unemployment rate, the lower the weekly sales

In [ ]:
# CPI vs. Weekly Sales
pearson_coef, p_value = pearsonr(df['CPI'], df['Weekly_Sales'])
print("The Pearson Correlation Coefficient is", pearson_coef, "with a P-value of P =", p_value)

**9.3 CPI vs. Weekly Sales**

In [ ]:
# Create a scatter plot with regression line
fig = px.scatter(df, x='CPI', y='Weekly_Sales', trendline='ols', 
                 title='Weekly Sales vs. Consumer Price Index', 
                 labels={'cpi': 'Consumer Price Index', 'weekly_sales': 'Weekly Sales'})

# Update trendline color
fig.data[1].line.color = 'red'

# Show plot
fig.show()

Conclusion:Since the p-value is < 0.001, the correlation between CPI and weekly sales is strong evidence that the correlation is significant

**9.4 Temperature vs. Weekly Sales**

In [ ]:
pearson_coef, p_value = pearsonr(df['Temperature'], df['Weekly_Sales'])
print("The Pearson Correlation Coefficient is", pearson_coef, "with a P-value of P =", p_value)

In [ ]:
# Create a scatter plot with regression line
fig = px.scatter(df, x='Temperature', y='Weekly_Sales', trendline='ols', 
                 title='Weekly Sales vs. Temperature', 
                 labels={'temperature': 'Temperature', 'weekly_sales': 'Weekly Sales'})

# Update trendline color
fig.data[1].line.color = 'red'

# Show plot
fig.show()

Conclusion:

* Since the p-value is < 0.05, the correlation between temperature and weekly sales is moderate evidence that the correlation is significant.
* Temperature seems like a good predictor of the weekly sales, The higher the Temperature rate, the lower the weekly sales.

**9.5 Calculate correlation coefficients between Weekly_Sales and other numerical variables to assess the strength and direction of relationships.**

In [ ]:
# Select numerical columns for correlation analysis
numerical_columns = ['Weekly_Sales', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']

# Calculate correlation matrix
correlation_matrix = df[numerical_columns].corr()

fig = go.Figure(data=go.Heatmap(
                    z=correlation_matrix.values, 
                    x=numerical_columns,
                    y=numerical_columns,
                    colorscale='RdYlBu',  
                    ))

fig.update_layout(title='Correlation Heatmap')
fig.show()


**9.5.1 Correlations with weekly sales:**

In [ ]:
corr = df[['Weekly_Sales', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']].corr()['Weekly_Sales'].sort_values(ascending = False)
corr = corr.to_frame()
corr.style.background_gradient(cmap="RdYlBu")

* Fuel Price and Weekly Sales: There’s a slight positive correlation, indicating that as fuel prices increase, weekly sales might also show a slight increase.
* Temperature and Weekly Sales: There’s a negative correlation, suggesting that higher temperatures might lead to a decrease in weekly sales.
* CPI and Weekly Sales: The negative correlation here implies that as the Consumer Price Index rises, weekly sales might decrease.
* Unemployment and Weekly Sales: Similarly, a negative correlation indicates that higher unemployment rates might be associated with lower weekly sales.

In summary, the dataset indicates that higher fuel prices have a very slight positive effect on weekly sales, while increased temperatures, a higher CPI, and higher unemployment rates might negatively impact weekly sales. However, all these effects are relatively small according to the correlation coefficients.

<h2 style = "color : red" >Project Questions</h2>
<br>


### 1. You are provided with the weekly sales data for their various outlets. Use statistical analysis, EDA, outlier analysis, and handle the missing values to come up with various insights that can give them a clear perspective on the following:
**a. If the weekly sales are affected by the unemployment rate, if yes - which stores
are suffering the most?**

Let just pick those who have less Weekly Sales. As it will be too lenghty to analyze 45 stores.
So what we do is we take 12 stores with least weekly_sales

In [ ]:
# Sort the sum of weekly sales in descending order and reset the index
sorted_sales = df.groupby('Store')['Weekly_Sales'].sum().sort_values(ascending=False).reset_index()
last_10_stores = sorted_sales.tail(12)['Store'].tolist()
print(last_10_stores)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Assuming df is your DataFrame containing the necessary data

# Initialize subplot grid
fig = make_subplots(rows=4, cols=3, subplot_titles=[
    f"Store {store_num}" for store_num in [42, 9, 29, 16, 37, 30, 3, 38, 36, 5, 44, 33]
])

# Iterate over each store and add a trace for the scatter plot
for i, store_num in enumerate([42, 9, 29, 16, 37, 30, 3, 38, 36, 5, 44, 33], start=1):
    store_data = df[df['Store'] == store_num]
    fig.add_trace(go.Scatter(x=store_data['Unemployment'], y=store_data['Weekly_Sales'],
                             mode='markers', name=f'Store {store_num}'),
                  row=(i - 1) // 3 + 1, col=(i - 1) % 3 + 1)

# Update layout to add titles
fig.update_layout(title_text="Weekly Sales vs Unemployment for Various Stores",
                  height=1200, width=1000)

fig.show()

* Now here we can clearly see that "Store 44" and "Store 42" is suffering the most by weekly sales
* These observations suggest that Store 42 and store 44 may require specific attention or strategies to mitigate the impact of unemployment rate fluctuations on its weekly sales performance. Meanwhile, for other stores, while there may still be some impact, it appears to be less severe or pronounced.

**b. If the weekly sales show a seasonal trend, when and what could be the reason?**

In [ ]:
sales_date = df[['Date','Weekly_Sales']]
sales_date.set_index('Date',inplace=True)
sales_date.head()

In [ ]:
# Create traces for each column in sales_date
traces = []
for column in sales_date.columns:
    trace = go.Scatter(x=sales_date.index, y=sales_date[column], mode='lines', name=column)
    traces.append(trace)

# Create the figure
fig = go.Figure(data=traces)

# Update layout
fig.update_layout(title="Weekly Sales over Time",
                  xaxis_title="Date",
                  yaxis_title="Weekly Sales",
                  legend_title="Stores",
                  width=1000, height=500)

# Show the plot
fig.show()


In [ ]:
# Lets check the sales by holiday season.

# Create trace
trace = go.Scatter(x=df['Date'], y=df['Holiday_Flag'], mode='lines', name='Holiday Flag')

# Create the figure
fig = go.Figure(data=trace)

# Update layout
fig.update_layout(title="Holiday Flag over Time",
                  xaxis_title="Date",
                  yaxis_title="Holiday Flag",
                  width=1000, height=500)

# Show the plot
fig.show()


* We can clearly observe a seasonality component in weekly sales, where sales tend to fluctuate cyclically throughout the year. However, towards the end of the year, there is a noticeable exponential increase in sales.
* This significant increase in sales towards the end of the year is primarily attributed to the holiday season, during which consumer spending typically rises.
* Notably, this spike in sales coincides with the holiday season, particularly during Christmas and New Year in North America, where Walmart holds significant prominence.
* It's common for brands, including Walmart, to offer various promotions and discounts during the holiday season. These promotional activities likely contribute to the sudden surge in sales observed at the end of the year.

**c. Does temperature affect the weekly sales in any manner?**

In [ ]:
# Calculate correlation coefficient between temperature and weekly sales
correlation = df['Temperature'].corr(df['Weekly_Sales'])
print(f'Correlation coefficient between temperature and weekly sales: {correlation}')

In [ ]:
import plotly.graph_objects as go

# Assuming data is your DataFrame containing the necessary data
# Assuming sales_date is your DataFrame containing sales data

# Create line plot for 'Temperature'
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=df['Date'], y=df['Temperature'], mode='lines', name='Temperature'))
fig1.update_layout(title="Temperature over Time",
                   xaxis_title="Date",
                   yaxis_title="Temperature",
                   width=1000, height=500)

# Create line plot for 'Weekly Sales'
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=sales_date.index, y=sales_date['Weekly_Sales'], mode='lines', name='Weekly Sales'))
fig2.update_layout(title="Weekly Sales over Time",
                   xaxis_title="Date",
                   yaxis_title="Weekly Sales",
                   width=1000, height=500)

# Show the plots
fig1.show()
fig2.show()


The only noted effect that can be seen is again of the holiday season. Holiday season are marked with winters and snow, that increases the needed clothing and stuff. Other than this there is no such clear trend of shopping related with temprature.

**d. How is the Consumer Price index affecting the weekly sales of various stores?**

In [ ]:
# Calculate correlation coefficient between temperature and weekly sales
correlation = df['CPI'].corr(df['Weekly_Sales'])
print(f'Correlation coefficient between CPI and weekly sales: {correlation}')

In [ ]:
import plotly.graph_objects as go

# Assuming data is your DataFrame containing the necessary data
# Assuming sales_date is your DataFrame containing sales data

# Create line plot for 'CPI'
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=df['Date'], y=df['CPI'], mode='lines', name='CPI'))
fig1.update_layout(title="Consumer Price Index (CPI) over Time",
                   xaxis_title="Date",
                   yaxis_title="CPI",
                   width=1000, height=500)
fig1.show()

# Create line plot for 'Weekly Sales'
fig2 = go.Figure()
for column in sales_date.columns:
    fig2.add_trace(go.Scatter(x=sales_date.index, y=sales_date[column], mode='lines', name=column))
fig2.update_layout(title="Weekly Sales over Time",
                   xaxis_title="Date",
                   yaxis_title="Weekly Sales",
                   width=1000, height=500)
fig2.show()


Although there is inflation over time represented by increasing CPI over the time period. There is no upward or downward trend followed by weekly sales.

**e. Top performing stores according to the historical data.**

In [ ]:
# Aggregate sales data for each store
total_sales = df.groupby('Store')['Weekly_Sales'].sum().reset_index()

# Rank stores based on total sales
total_sales_sorted = total_sales.sort_values(by='Weekly_Sales', ascending=False)

# Top performing stores
total_sales_sorted.head()

In [ ]:
fig = px.bar(total_sales_sorted, x='Store', y='Weekly_Sales', 
             labels={'Weekly_Sales': 'Total Sales'}, 
             title='Top Performing Stores')
fig.update_layout(xaxis_title='Store', yaxis_title='Total Sales')
fig.show()

Top performing 5 stores are --> store number 20, 4, 14, 13, 2

**f. The worst performing store, and how significant is the difference between the
highest and lowest performing stores.**

In [ ]:
total_sales_sorted.tail(5).sort_values('Weekly_Sales')

Worst performing stores are 33, 44, 5, 36, 38.

In [ ]:
import plotly.graph_objects as go

# Assuming Store20 and Store33 are your DataFrames containing the necessary data

# Create traces for Store20 and Store33
trace1 = go.Scatter(x=df[df['Store']==20]['Date'], y=df[df['Store']==20]['Weekly_Sales'], mode='lines', name='Store 20', line=dict(color='turquoise'))
trace2 = go.Scatter(x=df[df['Store']==33]['Date'], y=df[df['Store']==33]['Weekly_Sales'], mode='lines', name='Store 33', line=dict(color='chocolate'))

# Create subplot figure
fig = go.Figure([trace1, trace2])

# Update layout
fig.update_layout(title="Weekly Sales Comparison between best and worst performing store.png",
                  xaxis_title="Date",
                  yaxis_title="Weekly Sales",
                  width=1000, height=600)

# Show the plot
fig.show()

Significant difference in highest and lowest performing store.

In [ ]:
total_sales_sorted.set_index('Store', inplace=True)

In [ ]:
(total_sales_sorted.loc[33][0]/total_sales_sorted.loc[20][0])*100

Lowest performing store's sales only accounts for 12% of sales done by top performing store on average.

## Step:5 Predictive Modeling

#### Que:2 Use predictive modeling techniques to forecast the sales for each store for the next 12 weeks.

The choice of a model for continuous prediction depends on several factors, including the nature of your data, the presence of seasonality or trends, and the specific requirements of your problem.

Choosing the right model often involves experimentation and evaluation. It is good practice to try multiple models and compare their performance using appropriate evaluation metrics, such as R-squared, mean squared error (MSE), root mean squared error (RMSE), or mean absolute error (MAE), on a validation dataset. Additionally, consider the interpretability of the model and computational efficiency, especially for large-scale applications.

I have decided to compare Four ML models namely:

* Linear Regression
* Decision Tree
* Random Forest Regressor
* XGBOOST

We will compare these four models and pick the best perfoming model

### 5.1 Linear Regression

In [ ]:
df.columns

In [ ]:
# Split the data into features and target variable
X = df[['Store', 'Holiday_Flag', 'CPI', 'Day', 'Month', 'Year']]
y = df['Weekly_Sales']

In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Create a Linear Regression model
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()

In [ ]:
# Fit the model to the training data
linear_model.fit(X_train, y_train)

In [ ]:
# Get the coefficients of the Linear Regression model
coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': linear_model.coef_})
coefficients = coefficients.sort_values(by='Coefficient', ascending=False)
print("Linear Regression Coefficients:")
print()
print(coefficients)

These coefficients represent the relationship between each feature and the target variable, assuming all other features remain constant. Here’s what they mean:

* Holiday_Flag (66532.229815): When it’s a holiday, sales are expected to increase by approximately 66,532 units, holding all other features constant.
* Month (13726.510155): With each increase in month number (e.g., from January to February), sales are expected to increase by approximately 13,726 units.
Year (4488.717371): For each year, sales are expected to increase by approximately 4,488 units.
* Day (-1667.560022): With each passing day, sales are expected to decrease by approximately 1,667 units.
* CPI (-2088.070923): As the Consumer Price Index (CPI) increases by one unit, sales are expected to decrease by approximately 2,088 units.
* Store (-15719.395592): This might indicate that as the store number (perhaps as a proxy for location or other factors) increases, sales are expected to decrease by approximately 15,719 units.

Positive coefficients indicate a positive relationship with the target variable, while negative coefficients indicate a negative relationship. It’s important to note that these are statistical estimates and actual results may vary based on additional factors not included in the model.

In [ ]:
# Predict sales on the test set
y_pred_linear = linear_model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

print("Linear Regression R2 Score: ", r2_score(y_test, y_pred_linear))
print("Linear Regression MSE Score: ", mean_squared_error(y_test, y_pred_linear))
print("Linear Regression RMSE : ", sqrt(mean_squared_error(y_test, y_pred_linear)))

In [ ]:
# Create traces for actual and predicted sales
actual_trace = go.Scatter(x=df['Date'][len(df)-len(y_test):], y=y_test, mode='lines', name='Actual Weekly Sales', line=dict(color='blue'))
predicted_trace = go.Scatter(x=df['Date'][len(df)-len(y_test):], y=y_pred_linear, mode='lines', name='Predicted Weekly Sales (Linear Regression)', line=dict(color='red', dash='dash'))

# Create the figure
fig = go.Figure([actual_trace, predicted_trace])

# Update layout
fig.update_layout(xaxis_title='Date', yaxis_title='Weekly Sales', title='Actual vs Predicted Weekly Sales (Linear Regression)')

# Show the plot
fig.show()

### 5.2 Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Create an instance of DecisionTreeRegressor
dtree = DecisionTreeRegressor()

# Fit the model to the training data
dtree.fit(X_train, y_train)

In [ ]:
y_pred_dt = dtree.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

print("Decision Tree R2 Score: ", r2_score(y_test, y_pred_dt))
print("Decision Tree MSE Score: ", mean_squared_error(y_test, y_pred_dt))
print("Decision Tree RMSE : ", sqrt(mean_squared_error(y_test, y_pred_dt)))

In [ ]:
import plotly.graph_objects as go

# Create traces for actual and predicted sales
actual_trace = go.Scatter(x=df['Date'].iloc[-len(y_test):], y=y_test, mode='lines', name='Actual Weekly Sales', line=dict(color='blue'))
predicted_trace = go.Scatter(x=df['Date'].iloc[-len(y_test):], y=y_pred_dt, mode='lines', name='Predicted Weekly Sales (Decision Tree)', line=dict(color='red', dash='dash'))

# Create the figure
fig = go.Figure([actual_trace, predicted_trace])

# Update layout
fig.update_layout(xaxis_title='Date', yaxis_title='Weekly Sales', title='Actual vs Predicted Weekly Sales (Decision Tree)')

# Show the plot
fig.show()

### 5.3 Random Forest Regressor¶

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Create a Random Forest Regressor model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

In [ ]:
# Fit the model to the training data
rf_model.fit(X_train, y_train)

In [ ]:
# Predict sales on the test set
y_pred_rf = rf_model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

print("Random Forest Regressor R2 Score: ", r2_score(y_test, y_pred_rf))
print("Random Forest Regressor MSE Score: ", mean_squared_error(y_test, y_pred_rf))
print("Random Forest Regressor RMSE : ", sqrt(mean_squared_error(y_test, y_pred_rf)))

In [ ]:
import plotly.graph_objects as go

# Create traces for actual and predicted sales
actual_trace = go.Scatter(x=df['Date'].iloc[-len(y_test):], y=y_test, mode='lines', name='Actual Weekly Sales', line=dict(color='blue'))
predicted_trace = go.Scatter(x=df['Date'].iloc[-len(y_test):], y=y_pred_rf, mode='lines', name='Predicted Weekly Sales (Random Forest)', line=dict(color='red', dash='dash'))

# Create the figure
fig = go.Figure([actual_trace, predicted_trace])

# Update layout
fig.update_layout(xaxis_title='Date', yaxis_title='Weekly Sales', title='Actual vs Predicted Weekly Sales (Random Forest)')

# Show the plot
fig.show()

### 5.4 XG Boost

In [ ]:
from xgboost import XGBRegressor

# Create an instance of XGBRegressor
xg = XGBRegressor()

# Fit the model to the training data
xg.fit(X_train, y_train)

In [ ]:
y_pred_xg = xg.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

print("XG Boost R2 Score: ", r2_score(y_test, y_pred_xg))
print("XG Boost MSE Score: ", mean_squared_error(y_test, y_pred_xg))
print("XG Boost RMSE : ", sqrt(mean_squared_error(y_test, y_pred_xg)))

In [ ]:
# Create traces for actual and predicted sales
actual_trace = go.Scatter(x=df['Date'].iloc[-len(y_test):], y=y_test, mode='lines', name='Actual Weekly Sales', line=dict(color='blue'))
predicted_trace = go.Scatter(x=df['Date'].iloc[-len(y_test):], y=y_pred_xg, mode='lines', name='Predicted Weekly Sales (XG Boost)', line=dict(color='red', dash='dash'))

# Create the figure
fig = go.Figure([actual_trace, predicted_trace])

# Update layout
fig.update_layout(xaxis_title='Date', yaxis_title='Weekly Sales', title='Actual vs Predicted Weekly Sales (XG Boost)')

# Show the plot
fig.show()

### 5.5 Summary of Applied Predictive Modeling Techniques

In [ ]:
print("Linear Regression R2 Score: ", r2_score(y_test, y_pred_linear))
print("Linear Regression MSE Score: ", mean_squared_error(y_test, y_pred_linear))
print("Linear Regression RMSE : ", sqrt(mean_squared_error(y_test, y_pred_linear)))
print("Decision Tree R2 Score: ", r2_score(y_test, y_pred_dt))
print("Decision Tree MSE Score: ", mean_squared_error(y_test, y_pred_dt))
print("Decision Tree RMSE : ", sqrt(mean_squared_error(y_test, y_pred_dt)))
print("Random Forest Regressor R2 Score: ", r2_score(y_test, y_pred_rf))
print("Random Forest Regressor MSE Score: ", mean_squared_error(y_test, y_pred_rf))
print("Random Forest Regressor RMSE : ", sqrt(mean_squared_error(y_test, y_pred_rf)))
print("XG Boost R2 Score: ", r2_score(y_test, y_pred_xg))
print("XG Boost MSE Score: ", mean_squared_error(y_test, y_pred_xg))
print("XG Boost RMSE : ", sqrt(mean_squared_error(y_test, y_pred_xg)))

The XG Boost model outperforms the others in terms of R2 Score, MSE Score, and RMSE. It shows the highest R2 Score, indicating better fit to the data, and the lowest MSE and RMSE, indicating lower prediction errors.

Therefore, we should choose the XG Boost model for sales forecasting because it provides the most accurate predictions compared to the other models.

**5.5.1 Predicting weekly_sales for 1st row using XGBOOST**

In [ ]:
X.head(1)

In [ ]:
y.head(1)

In [ ]:
xg.predict(pd.DataFrame([[1,0,211.096358,5,2,2010]]))[0]

Here, We're using XGBoost to predict the weekly sales for the first row, and it's performing exceptionally well in making accurate predictions

## 5.6 Sales forecasting for the next 12 weeks using XGBoost for all 45 stores.

In [ ]:
df['Date'].max()

In [ ]:
latest_week_date = pd.Timestamp('2012-10-26')
next_sunday = latest_week_date + pd.DateOffset(days=(6 - latest_week_date.dayofweek) + 1)  # Get the next Sunday
date_range = pd.date_range(start=next_sunday, periods=12, freq='W-FRI')  # Start from Friday instead of Sunday
date_range

In [ ]:
# Assuming the last week in your dataset is the most recent week
latest_week_date = df['Date'].max()

# Creating a DataFrame with dates for the next 12 weeks
next_12_weeks = pd.date_range(start=latest_week_date, periods=12, freq='W')

latest_week_date = pd.Timestamp('2012-10-26')
next_sunday = latest_week_date + pd.DateOffset(days=(6 - latest_week_date.dayofweek) + 1)  # Get the next Sunday
next_12_weeks = pd.date_range(start=next_sunday, periods=12, freq='W-FRI')  # Start from Friday instead of Sunday


# Creating a DataFrame to hold predictions
forecast_df = pd.DataFrame(index=next_12_weeks, columns=range(1, 46))

# Making predictions for next 12 weeks for each store
for store_num in range(1, 46):
    store = df[df['Store'] == 1][['Store', 'Holiday_Flag', 'CPI', 'Weekly_Sales', 'Date']]
    future_data = pd.DataFrame({
        'Store': [store_num] * 12,
        'Holiday_Flag': [0] * 12,  # Assuming no holidays in the next 12 weeks
        'CPI': [store.CPI.mean()] * 12,
        'Day': next_12_weeks.day,
        'Month': next_12_weeks.month,
        'Year': next_12_weeks.year,
        'Date': next_12_weeks
    })

    # Making predictions for next 12 weeks for the current store
    future_predictions = xg.predict(future_data.drop(columns=['Date']))

    # Storing predictions in the forecast DataFrame
    forecast_df[store_num] = future_predictions

# Round the forecasted sales values to two decimal points
forecast_df = forecast_df.round(2)

# Transpose the forecast DataFrame for better visualization
forecast_df = forecast_df.transpose()

# Rename the index to 'Store'
forecast_df.index.name = 'Store'

In [ ]:
# Transpose the forecast DataFrame for better visualization
forecast_df = forecast_df.transpose()

# Round the forecasted sales values to two decimal points
forecast_df = forecast_df.round(2)

pd.DataFrame(forecast_df)

Above matrix is the forecast for all the stores for next 12 weeks.

## 5.7 Actual vs Forecasted Graph for 5 Stores

In [ ]:
for i in range(1,6):
    # Selecting store Individually 
    store = df[df['Store'] == i][['Store', 'Holiday_Flag', 'CPI', 'Weekly_Sales', 'Date']]

    # Extract day,mont,year component from the 'Date' column
    store['Day'] = store['Date'].dt.day
    store['Month'] = store['Date'].dt.month
    store['Year'] = store['Date'].dt.year

    # Only date column from individually selected store
    only_date = pd.DataFrame(store['Date'])
    
    # Only date column from individually selected store
    only_weekly_sales = pd.DataFrame(store['Weekly_Sales'])

    # Drop Date and Weekly_Sales column from store dataframe
    store = store.drop(columns=['Date','Weekly_Sales'])

    # Getting forecasting dates for next 12 weeks
    latest_week_date = pd.Timestamp('2012-10-26')
    next_sunday = latest_week_date + pd.DateOffset(days=(6 - latest_week_date.dayofweek) + 1)  # Get the next Sunday
    date_range = pd.date_range(start=next_sunday, periods=12, freq='W-FRI')  # Start from Friday instead of Sunday
    next_date = pd.DataFrame(date_range)
    next_date = next_date.rename(columns={0: 'Date'})

    # Concatenate actual+next 12 weeks date
    old_plus_new_date = pd.concat([only_date,next_date], axis = 0)

    # Inputs for predicting next 12 weeks weekly_sales
    data = {'Store': [i] * 12,
            'Holiday_Flag': [0] * 12,
            'CPI': [store.CPI.mean()] * 12,
            'Day': next_12_weeks.day,
            'Month': next_12_weeks.month,
            'Year': next_12_weeks.year}

    # Make dataframe considering above Data 
    forecast_df = pd.DataFrame(data)
    forecast_df

    # Concatenate Individual store plus 'Inputs for predicting next 12 weeks weekly_sales'
    whole = pd.concat([store,forecast_df,], axis = 0)

    # Predicting weekly_sales for historical plus future 
    all = xg.predict(whole)

    # Create traces for actual and predicted sales
    actual_trace = go.Scatter(x=df['Date'].iloc[-len(store):], y=only_weekly_sales.Weekly_Sales, mode='lines', name='Actual Weekly Sales', line=dict(color='blue'))
    predicted_trace = go.Scatter(x=old_plus_new_date['Date'].iloc[-len(old_plus_new_date):], y=all, mode='lines', name='Predicted Weekly Sales (XG Boost)', line=dict(color='red', dash='dash'))

    # Create the figure
    fig = go.Figure([actual_trace, predicted_trace])

    # Update layout
    fig.update_layout(xaxis_title='Date', yaxis_title='Weekly Sales', title=f'Actual vs Predicted Weekly Sales for store {i} (XG Boost)')

    # Show the plot
    fig.show()

So, here we have plotted graphs for the actual versus predicted weekly sales for 5 stores. If we want, we can generate similar plots for other stores by adjusting the specified range accordingly.

The variations between the actual and predicted weekly sales could indeed be influenced by factors like CPI and holiday flags. In our prediction model, we've assumed that CPI remains constant at the mean value for each store and that there are no holidays (holiday_flag = 0) for the next 12 weeks. However, in reality, there might be fluctuations in CPI and holidays such as Christmas and New Year during the upcoming weeks, which can affect sales.

## Step:6  Model Saving and Loading for Scalability and Reproducibility

#### 6.1 Saving the model

In [ ]:
# Step 1: Saving the model
model_file_path = '../models/wallmart.pkl'

# Save the model to the specified file path
joblib.dump(xg, model_file_path)

#### 6.2 Loading the model

In [ ]:
# Step 2: Loading the model
loaded_model = joblib.load(model_file_path)

#### 6.3 Perform predictions using the loaded model

In [ ]:
# Step 3: Perform predictions using the loaded model
predictions = loaded_model.predict(pd.DataFrame([[1,0,211.096358,5,2,2010]]))[0]
print(predictions)

## Step: 7 Conclusion

In conclusion, this project aimed to forecast sales for multiple stores over the next 12 weeks
using predictive modeling techniques. Through comprehensive data analysis, model
development, and evaluation, valuable insights have been uncovered that can inform
strategic decision-making and drive business growth.

**Key Findings:**

`1. Impact of Predictor Variables:` The analysis revealed the significant influence of
predictor variables such as holiday flags, consumer price index (CPI), and
unemployment rates on weekly sales. Understanding these factors is crucial for
anticipating sales trends and optimizing resource allocation.                 
`2. Seasonal Trends and Patterns:` Seasonal fluctuations in sales were observed,
highlighting the importance of accounting for temporal patterns in forecasting
models. By incorporating seasonal adjustments, businesses can better align their
operations with demand fluctuations throughout the year.              
`3. Model Performance:` Evaluation of various predictive modeling techniques
demonstrated the superiority of advanced algorithms such as Random Forest and
XGBOOST in forecasting sales accurately. These models outperformed simpler
techniques and provided more robust predictions.            
`4. Business Insights for Decision-making:` The insights generated from sales forecasting
can guide strategic decision-making in areas such as inventory management,
marketing strategies, and resource allocation. By leveraging predictive analytics,
businesses can optimize operations, reduce costs, and improve customer satisfaction            